# Create A Sample Manifest File for STAR-Fusion NF Workflow

### Jenny Smith 
### Oct 14, 2020

In [1]:
import os
import pathlib
import socket
import numpy as np
import pandas as pd
import re
import boto3
import botocore
import create_sample_sheet


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# https://realpython.com/python-pathlib/
host = socket.gethostname()
TARGET="workingDir/TARGET/AML_TARGET"
BEAT_AML="workingDir/BEAT_AML"

if re.search("MacBook|mbp.+fhcrc.org", host):
    ROOT_DIR=pathlib.Path("/Users/work/fast_drive/")

elif re.search("gizmo|rhino", host):
    ROOT_DIR=pathlib.Path("/fh/fast/meshinchi_s/")

TARGET=ROOT_DIR.joinpath(TARGET)
BEAT_AML=ROOT_DIR.joinpath(BEAT_AML)

In [3]:
pathlib.Path().cwd()

PosixPath('/Users/work/batch_pipeline/python_code')

# 1. Read in the Manifest file

In [4]:
# [{print(f)}for file in TARGET.joinpath("SequencingDataMatrix/").iterdir()]

manifest_file = BEAT_AML.joinpath("RNA/mRNAseq/analysis/2020.06.20_GDC_Download/BEAT_AML_STAR-aligner_GCD_Data_Manifest_with_CDE.csv")


manifest = pd.read_csv(manifest_file, 
                       engine='python')
manifest.fillna(value="",inplace=True) 
manifest.head()
manifest.shape

,PatientID,PatientId,LLS_PatientID,LLS_SampleID,SampleID,entity_submitter_id,file_name,OverallSurvival_VIZOME,VitalStatus_VIZOME,vitalStatus_SuppTable,...,STAG2_SuppTable,STAT3_SuppTable,SUZ12_SuppTable,TCL1A_SuppTable,TET2_SuppTable,TP53_SuppTable,TYK2_SuppTable,U2AF1_SuppTable,WT1_SuppTable,ZRSR2_SuppTable
0,2262,1981,1981,15-00147,BA3025,aq-BA3025R,90e79303-22a7-4c03-ad58-a41af7af237e.rna_seq.g...,843,Dead,Dead,...,,,,,,,,,,
1,2200,1757,1757,15-00081,BA2415,aq-BA2415R,e9332d30-3f0f-422a-abb2-d10d9a1e44f7.rna_seq.g...,443,Dead,Dead,...,,,,,,,,,,
2,2299,,4349,17-00047,BA3034,aq-BA3034R,fae6404d-68c4-4e13-ade2-dc0908f2c074.rna_seq.g...,"Healthy, Individual BM MNC","Healthy, Individual BM MNC",,...,,,,,,,,,,
3,2448,1646,1646,14-00425,BA2811,aq-BA2811R,ce88002b-b22a-452a-a1d3-b8395dc118f1.rna_seq.g...,461,Alive,Alive,...,,,,,,negative,,,negative,
4,2093,,4350,17-00048,BA2607,aq-BA2607R,7824c0f0-85c6-44e0-9162-8865c9e1cb74.rna_seq.g...,"Healthy, Individual BM MNC","Healthy, Individual BM MNC",,...,,,,,,,,,,


(510, 248)

In [5]:
# dir(manifest.SampleID)
manifest.SampleID.duplicated().value_counts() #no duplicates 

False    510
Name: SampleID, dtype: int64

# 2. Run the Sample Sheet Script

In [6]:
bucket="fh-pi-meshinchi-s"
prefix="SR/BEAT_AML/BAM/"
?create_sample_sheet.create_sample_sheet
# ?create_sample_sheet.create_presigned_url

Signature:
create_sample_sheet.create_sample_sheet(
    bucket_name,
    prefix_name,
    filetype='fastq',
    samples='',
    filename='sample_sheet.txt',
    write=True,
)
Docstring:
A function to query an S3 bucket, list its objects, and filter the files by sample IDs. 

bucket_name: is a string. Example: "fh-pi-my-bucket"
prefix_name: is a string. Need trailing slash. Example: "SR/myfiles/"
filetype: is a srtring. One of "fastq" or "bam"
File:      ~/batch_pipeline/python_code/create_sample_sheet.py
Type:      function


In [7]:
#create an input string for filtering the manifest file
samples = " ".join(manifest.SampleID.tolist())
type(samples)

str

In [ ]:
create_sample_sheet.create_sample_sheet(bucket_name=bucket, 
                                        prefix_name=prefix,
                                        filetype="bam",  
                                        samples=samples,
                                        filename="../sample_sheets/BEAT_AML_sample_sheet.txt")